# TP: implementation de  quelques algorithmes populaires (SVM et AD)

Explorons une application sur le dataset [auto MGP](https://archive.ics.uci.edu/ml/datasets/Auto%2BMPG): contient des données sur des modèles de voitures

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv('https://filedn.eu/lefeldrXcsSFgCcgc48eaLY/datasets/regression/auto-mpg.csv',index_col='car name')
data.head()

In [ ]:
data['origin'].value_counts()

- `mpg`: miles par galion
- `cylinder`: nombre de cylindres
- `displacement`: volume des pistons (en cc)
- `horsepower`: chevaux fiscaux
- `weight`: poids
- `acceleration`: acceleration (en s)

**Le but est de prédire la variable `mpg` en fonction des autres features**

## Préparation des données

### Nettoyer des données

- Explorer les variables avec les fonctions `describe` et `info`.
- Y a-t-il des valeurs manquantes dans les données ?
- Toutes les variables sont-elles du bon type ?
- Comment allez-vous traiter les anomalies dans la variable `horsepower` ?
- Quelles sont les variables fortement corrélées à la variable cible ?


In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data['horsepower'].unique() #value_counts()

In [ ]:
data.horsepower[data.horsepower == '?'].count()

Les valeurs manquantes étant peu nombreuses, on choisit de les imputer par un knn imputer : 

In [ ]:
data['horsepower'] = data['horsepower'].replace(to_replace='?', value=np.nan)

In [ ]:
data.horsepower[data.horsepower == '?'].count()

In [ ]:
data.isnull().sum()

In [ ]:
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=5, missing_values=np.nan)
data.horsepower = knn_imputer.fit_transform(data.horsepower.values.reshape(-1,1))

In [ ]:
data.isnull().sum()

In [ ]:
mask = np.triu(np.ones_like(data.corr(), dtype=bool)) # Masque triangulaire
fig = plt.figure(figsize=(9, 9))
ax = sns.heatmap(data.corr(), mask=mask, vmin=-1, vmax=1,
                 square=True,linewidths=0.9,
                 annot=True, annot_kws={'fontsize':'large'}, cmap='BrBG')
ax.set_title('Correlation Heatmap')
ax.set_xlabel('', fontsize='25');
plt.xticks(rotation = 45);

### Préparer les données pour la modélisation

- Séparer la variable cible des variables explicatives.  
- Séparer les données d'entraînement des données de test.
- Créer une copie normalisée des données : `scaled_X_train` et `scaled_X_test`


In [ ]:
y = data.mpg
X = data.drop(columns='mpg')
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1,1))
y_train_scaled = np.ravel(y_train_scaled)

X_test_scaled = scaler.fit_transform(X_test)
y_test_scaled = scaler.fit_transform(y_test.values.reshape(-1,1))
y_test_scaled = np.ravel(y_test_scaled)

## Modélisation 

- Mettre en place des modèles SVM et AD adaptés à la tâche voulue.
- Entraîner les modèles sur les données d'entraînement.
- Utiliser les données normalisées dans le cas de modèles sensibles à l'échelle.
- Penser à utiliser la validation croisée

## Evaluation des modèles

- Calculer les métriques adéquates.
- Tester le sur-apprentissage et le sous-apprentissage.(à voir demain)
- Optimiser les paramètres des modèles

**SVM**

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

model = SVR()
param_grid = {'C': [0.1, 1, 10, 100],'kernel': ['linear', 'rbf', 'poly']}

grid_search = GridSearchCV(model, param_grid=param_grid)
grid_search.fit(X_train_scaled,y_train_scaled)


In [ ]:
print(grid_search.best_params_)

In [ ]:
model = SVR(kernel='rbf', C=1)
param_grid = {'gamma': [0.01, 0.1, 1, 10], 'epsilon': [0.01, 0.1, 0.5, 1]}

grid_search = GridSearchCV(model, param_grid=param_grid)
grid_search.fit(X_train_scaled,y_train_scaled)

In [ ]:
print(grid_search.best_params_)

In [ ]:
best_svr = grid_search.best_estimator_
best_svr

In [ ]:
from sklearn.metrics import r2_score

y_pred = best_svr.predict(X_test_scaled)
r2_svm= r2_score(y_pred, y_test_scaled)
r2_svm

-------------------------

**Arbres de décision**

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model_DT = DecisionTreeRegressor(random_state=40)

param_grid= {'max_depth': [4, 5, 6,7,8,9,10, 20, 100]}

grid_search = GridSearchCV(model_DT, param_grid=param_grid)
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_params_)

In [ ]:
best_DT = grid_search.best_estimator_
y_pred = best_DT.predict(X_test)
r2_DT = r2_score(y_pred, y_test)
r2_DT

**Régression linéaire**

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train_scaled)

y_pred = linreg.predict(X_test_scaled)
r2_rl = r2_score(y_pred, y_test_scaled)
r2_rl

**Quel modèle offre la meilleure performance sur ces données ?**

In [ ]:
score_r2_ = {'SVR':r2_svm, 'DecisionTreeRegressor' : r2_DT, 'LinearRegression': r2_rl}

score_r2_tries = sorted(score_r2.items(), key=lambda item: item[1], reverse=True)

best_model, best_score = score_r2_tries[0]
print(f'le meilleur modele:, {best_model}'  )
print(f'le meilleur score:, {best_score}'  )